In [173]:
import torch 
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
data_dir = "/home/lbenboudiaf/Bureau/ImageClassificationWithDeepLearning/GroceryStoreDataset/dataset/dataset2"
model_name = "squeezenet"
num_classes = 81
batch_size = 8
num_epochs = 1000 #Valeur Initiale par défaut comme dans le TP du Prof 
feature_extract = True

In [174]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    #Tableau "losses" afin de stocker toutes les valeurs calculer.
    losses_train = []
    losses_val = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # récupéré le model et calculer sa fonction loss 
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase, Backward 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            if phase == 'train': 
                losses_train.append(epoch_loss)
            else:
                losses_val.append(epoch_loss)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, losses_train, losses_val

In [175]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [176]:
#Definition du model Squeezenet.
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "squeezenet":
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
    else:
        print("Invalid model name")
        exit()

    return model_ft, input_size

# Initialisé le model pour ce dataset 
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

In [177]:
import os
# Augmentaion et normalisation de données pour le jeux d'apprentissage.  
# Normalisation des jeux de test et de validation 
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Generation de jeux d'apprentissage et de validation 
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
# Creation des dataloaders pour jeux d'apprentissage et de validation. 
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [178]:
# Envoyer le model au GPU
model_ft = model_ft.to(device)
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observation des paramètres sont optimisé 
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [179]:
import copy 
#fonction de loss
criterion = nn.CrossEntropyLoss()

# Envoyer le jeux de d'apprentissage et de validation, EVALUATION DU Model. 
# Premier essaye avec 15 epoche, déduction le nombre optimale d'époche = 5. 
# Deuxième essaye meilleur resultat
model_ft, hist, loss_train, loss_val = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs, is_inception=(model_name=="inception"))


Epoch 0/999
----------
train Loss: 2.3405 Acc: 0.4198
val Loss: 1.4187 Acc: 0.5946
Epoch 1/999
----------
train Loss: 0.4552 Acc: 0.8148
val Loss: 0.4906 Acc: 0.8108
Epoch 2/999
----------
train Loss: 0.3162 Acc: 0.8889
val Loss: 0.6698 Acc: 0.7669
Epoch 3/999
----------
train Loss: 0.2158 Acc: 0.9136
val Loss: 0.7195 Acc: 0.7872
Epoch 4/999
----------
train Loss: 0.2363 Acc: 0.9136
val Loss: 0.5966 Acc: 0.8243
Epoch 5/999
----------
train Loss: 0.1603 Acc: 0.9259
val Loss: 0.5755 Acc: 0.8446
Epoch 6/999
----------
train Loss: 0.0446 Acc: 0.9753
val Loss: 0.5591 Acc: 0.8581
Epoch 7/999
----------
train Loss: 0.1630 Acc: 0.9630
val Loss: 0.9996 Acc: 0.7331
Epoch 8/999
----------
train Loss: 0.2054 Acc: 0.9383
val Loss: 0.7566 Acc: 0.8074
Epoch 9/999
----------
train Loss: 0.0552 Acc: 0.9753
val Loss: 0.6588 Acc: 0.8311
Epoch 10/999
----------
train Loss: 0.1089 Acc: 0.9753
val Loss: 0.7000 Acc: 0.8311
Epoch 11/999
----------
train Loss: 0.0917 Acc: 0.9753
val Loss: 0.6226 Acc: 0.8480
Ep

val Loss: 0.7877 Acc: 0.8514
Epoch 98/999
----------
train Loss: 0.0213 Acc: 0.9877
val Loss: 0.7413 Acc: 0.8750
Epoch 99/999
----------
train Loss: 0.0252 Acc: 0.9877
val Loss: 0.7554 Acc: 0.8682
Epoch 100/999
----------
train Loss: 0.0311 Acc: 0.9753
val Loss: 0.8073 Acc: 0.8784
Epoch 101/999
----------
train Loss: 0.0040 Acc: 1.0000
val Loss: 0.8225 Acc: 0.8682
Epoch 102/999
----------
train Loss: 0.0007 Acc: 1.0000
val Loss: 0.8167 Acc: 0.8682
Epoch 103/999
----------
train Loss: 0.0057 Acc: 1.0000
val Loss: 0.8040 Acc: 0.8716
Epoch 104/999
----------
train Loss: 0.0006 Acc: 1.0000
val Loss: 0.8043 Acc: 0.8716
Epoch 105/999
----------
train Loss: 0.0368 Acc: 0.9753
val Loss: 0.9802 Acc: 0.8446
Epoch 106/999
----------
train Loss: 0.0170 Acc: 0.9877
val Loss: 1.3659 Acc: 0.7973
Epoch 107/999
----------
train Loss: 0.0251 Acc: 0.9877
val Loss: 1.1002 Acc: 0.8108
Epoch 108/999
----------
train Loss: 0.0174 Acc: 0.9877
val Loss: 0.8591 Acc: 0.8581
Epoch 109/999
----------
train Loss: 0

train Loss: 0.0055 Acc: 1.0000
val Loss: 1.1187 Acc: 0.8378
Epoch 195/999
----------
train Loss: 0.0039 Acc: 1.0000
val Loss: 1.2578 Acc: 0.8243
Epoch 196/999
----------
train Loss: 0.0918 Acc: 0.9630
val Loss: 0.9819 Acc: 0.8446
Epoch 197/999
----------
train Loss: 0.0468 Acc: 0.9877
val Loss: 0.9919 Acc: 0.8446
Epoch 198/999
----------
train Loss: 0.0013 Acc: 1.0000
val Loss: 1.0598 Acc: 0.8378
Epoch 199/999
----------
train Loss: 0.0140 Acc: 0.9877
val Loss: 1.0230 Acc: 0.8412
Epoch 200/999
----------
train Loss: 0.0061 Acc: 1.0000
val Loss: 0.9700 Acc: 0.8412
Epoch 201/999
----------
train Loss: 0.0091 Acc: 1.0000
val Loss: 0.9863 Acc: 0.8412
Epoch 202/999
----------
train Loss: 0.0003 Acc: 1.0000
val Loss: 0.9989 Acc: 0.8345
Epoch 203/999
----------
train Loss: 0.0228 Acc: 0.9877
val Loss: 0.9841 Acc: 0.8378
Epoch 204/999
----------
train Loss: 0.0038 Acc: 1.0000
val Loss: 0.9590 Acc: 0.8446
Epoch 205/999
----------
train Loss: 0.0012 Acc: 1.0000
val Loss: 0.9614 Acc: 0.8446
Epoch

train Loss: 0.0120 Acc: 1.0000
val Loss: 1.0642 Acc: 0.8378
Epoch 292/999
----------
train Loss: 0.0297 Acc: 0.9877
val Loss: 1.0367 Acc: 0.8277
Epoch 293/999
----------
train Loss: 0.0041 Acc: 1.0000
val Loss: 1.0403 Acc: 0.8311
Epoch 294/999
----------
train Loss: 0.1414 Acc: 0.9753
val Loss: 1.0901 Acc: 0.8345
Epoch 295/999
----------
train Loss: 0.0135 Acc: 0.9877
val Loss: 1.1231 Acc: 0.8277
Epoch 296/999
----------
train Loss: 0.0026 Acc: 1.0000
val Loss: 1.1312 Acc: 0.8209
Epoch 297/999
----------
train Loss: 0.0161 Acc: 0.9877
val Loss: 1.1724 Acc: 0.8243
Epoch 298/999
----------
train Loss: 0.0047 Acc: 1.0000
val Loss: 1.2337 Acc: 0.8209
Epoch 299/999
----------
train Loss: 0.0052 Acc: 1.0000
val Loss: 1.2221 Acc: 0.8243
Epoch 300/999
----------
train Loss: 0.0003 Acc: 1.0000
val Loss: 1.2119 Acc: 0.8243
Epoch 301/999
----------
train Loss: 0.0328 Acc: 0.9753
val Loss: 1.1988 Acc: 0.8243
Epoch 302/999
----------
train Loss: 0.0021 Acc: 1.0000
val Loss: 1.2502 Acc: 0.8209
Epoch

train Loss: 0.0214 Acc: 0.9877
val Loss: 1.0500 Acc: 0.8412
Epoch 389/999
----------
train Loss: 0.0023 Acc: 1.0000
val Loss: 1.0378 Acc: 0.8378
Epoch 390/999
----------
train Loss: 0.0015 Acc: 1.0000
val Loss: 1.0271 Acc: 0.8412
Epoch 391/999
----------
train Loss: 0.0005 Acc: 1.0000
val Loss: 1.0229 Acc: 0.8378
Epoch 392/999
----------
train Loss: 0.0028 Acc: 1.0000
val Loss: 1.0176 Acc: 0.8378
Epoch 393/999
----------
train Loss: 0.0053 Acc: 1.0000
val Loss: 1.0168 Acc: 0.8378
Epoch 394/999
----------
train Loss: 0.0512 Acc: 0.9877
val Loss: 1.1318 Acc: 0.8345
Epoch 395/999
----------
train Loss: 0.0105 Acc: 1.0000
val Loss: 1.2372 Acc: 0.8311
Epoch 396/999
----------
train Loss: 0.0104 Acc: 1.0000
val Loss: 1.1024 Acc: 0.8345
Epoch 397/999
----------
train Loss: 0.0022 Acc: 1.0000
val Loss: 1.0880 Acc: 0.8480
Epoch 398/999
----------
train Loss: 0.0014 Acc: 1.0000
val Loss: 1.0930 Acc: 0.8412
Epoch 399/999
----------
train Loss: 0.0016 Acc: 1.0000
val Loss: 1.0972 Acc: 0.8412
Epoch

...

train Loss: 0.0273 Acc: 0.9877
val Loss: 2.0229 Acc: 0.7770
Epoch 583/999
----------
train Loss: 0.1195 Acc: 0.9506
val Loss: 3.9950 Acc: 0.6453
Epoch 584/999
----------
train Loss: 0.1498 Acc: 0.9506
val Loss: 4.0112 Acc: 0.6486
Epoch 585/999
----------
train Loss: 0.0091 Acc: 1.0000
val Loss: 3.7555 Acc: 0.6757
Epoch 586/999
----------
train Loss: 0.0001 Acc: 1.0000
val Loss: 3.5818 Acc: 0.6791
Epoch 587/999
----------
train Loss: 0.0045 Acc: 1.0000
val Loss: 3.5024 Acc: 0.6926
Epoch 588/999
----------
train Loss: 0.0012 Acc: 1.0000
val Loss: 3.4819 Acc: 0.6926
Epoch 589/999
----------
train Loss: 0.0077 Acc: 1.0000
val Loss: 3.5970 Acc: 0.6858
Epoch 590/999
----------
train Loss: 0.0129 Acc: 1.0000
val Loss: 3.7831 Acc: 0.6689
Epoch 591/999
----------
train Loss: 0.0544 Acc: 0.9753
val Loss: 3.4762 Acc: 0.6993
Epoch 592/999
----------
train Loss: 0.1522 Acc: 0.9753
val Loss: 3.4762 Acc: 0.6959
Epoch 593/999
----------
train Loss: 0.0601 Acc: 0.9877
val Loss: 4.0427 Acc: 0.6689
Epoch

train Loss: 0.0118 Acc: 1.0000
val Loss: 1.4839 Acc: 0.8007
Epoch 680/999
----------
train Loss: 0.0048 Acc: 1.0000
val Loss: 1.5923 Acc: 0.7872
Epoch 681/999
----------
train Loss: 0.0017 Acc: 1.0000
val Loss: 1.6445 Acc: 0.7770
Epoch 682/999
----------
train Loss: 0.0001 Acc: 1.0000
val Loss: 1.6619 Acc: 0.7703
Epoch 683/999
----------
train Loss: 0.0075 Acc: 1.0000
val Loss: 1.6865 Acc: 0.7669
Epoch 684/999
----------
train Loss: 0.0249 Acc: 0.9877
val Loss: 1.6114 Acc: 0.7804
Epoch 685/999
----------
train Loss: 0.0307 Acc: 0.9877
val Loss: 1.5019 Acc: 0.7939
Epoch 686/999
----------
train Loss: 0.0006 Acc: 1.0000
val Loss: 1.4545 Acc: 0.8142
Epoch 687/999
----------
train Loss: 0.0657 Acc: 0.9877
val Loss: 1.4147 Acc: 0.8243
Epoch 688/999
----------
train Loss: 0.0000 Acc: 1.0000
val Loss: 1.3921 Acc: 0.8311
Epoch 689/999
----------
train Loss: 0.0018 Acc: 1.0000
val Loss: 1.3892 Acc: 0.8311
Epoch 690/999
----------
train Loss: 0.0113 Acc: 1.0000
val Loss: 1.4015 Acc: 0.8277
Epoch

train Loss: 0.0003 Acc: 1.0000
val Loss: 1.5386 Acc: 0.8074
Epoch 777/999
----------
train Loss: 0.0077 Acc: 1.0000
val Loss: 1.5374 Acc: 0.8108
Epoch 778/999
----------
train Loss: 0.0019 Acc: 1.0000
val Loss: 1.4914 Acc: 0.8074
Epoch 779/999
----------
train Loss: 0.0016 Acc: 1.0000
val Loss: 1.4499 Acc: 0.8074
Epoch 780/999
----------
train Loss: 0.0086 Acc: 1.0000
val Loss: 1.4660 Acc: 0.8074
Epoch 781/999
----------
train Loss: 0.0136 Acc: 0.9877
val Loss: 1.4728 Acc: 0.7973
Epoch 782/999
----------
train Loss: 0.0123 Acc: 0.9877
val Loss: 1.4792 Acc: 0.7973
Epoch 783/999
----------
train Loss: 0.0085 Acc: 1.0000
val Loss: 1.4975 Acc: 0.7905
Epoch 784/999
----------
train Loss: 0.0121 Acc: 0.9877
val Loss: 1.3695 Acc: 0.8142
Epoch 785/999
----------
train Loss: 0.0002 Acc: 1.0000
val Loss: 1.2929 Acc: 0.8209
Epoch 786/999
----------
train Loss: 0.0012 Acc: 1.0000
val Loss: 1.2725 Acc: 0.8209
Epoch 787/999
----------
train Loss: 0.0460 Acc: 0.9753
val Loss: 1.2717 Acc: 0.8176
Epoch

train Loss: 0.0199 Acc: 0.9877
val Loss: 1.5783 Acc: 0.7939
Epoch 874/999
----------
train Loss: 0.0112 Acc: 0.9877
val Loss: 1.5467 Acc: 0.7939
Epoch 875/999
----------
train Loss: 0.0351 Acc: 0.9877
val Loss: 1.5220 Acc: 0.8007
Epoch 876/999
----------
train Loss: 0.0064 Acc: 1.0000
val Loss: 1.4951 Acc: 0.8007
Epoch 877/999
----------
train Loss: 0.0000 Acc: 1.0000
val Loss: 1.4758 Acc: 0.8041
Epoch 878/999
----------
train Loss: 0.0043 Acc: 1.0000
val Loss: 1.4640 Acc: 0.8007
Epoch 879/999
----------
train Loss: 0.0405 Acc: 0.9877
val Loss: 1.5226 Acc: 0.8142
Epoch 880/999
----------
train Loss: 0.0782 Acc: 0.9877
val Loss: 1.5043 Acc: 0.8176
Epoch 881/999
----------
train Loss: 0.0065 Acc: 1.0000
val Loss: 1.4268 Acc: 0.8007
Epoch 882/999
----------
train Loss: 0.0552 Acc: 0.9877
val Loss: 1.4240 Acc: 0.8041
Epoch 883/999
----------
train Loss: 0.0476 Acc: 0.9877
val Loss: 1.4184 Acc: 0.8074
Epoch 884/999
----------
train Loss: 0.0116 Acc: 0.9877
val Loss: 1.4287 Acc: 0.8041
Epoch

train Loss: 0.0502 Acc: 0.9877
val Loss: 1.3461 Acc: 0.8041
Epoch 971/999
----------
train Loss: 0.0009 Acc: 1.0000
val Loss: 1.3821 Acc: 0.8041
Epoch 972/999
----------
train Loss: 0.0392 Acc: 0.9877
val Loss: 1.3915 Acc: 0.8041
Epoch 973/999
----------
train Loss: 0.0359 Acc: 0.9877
val Loss: 1.3394 Acc: 0.7939
Epoch 974/999
----------
train Loss: 0.0943 Acc: 0.9877
val Loss: 1.3202 Acc: 0.8041
Epoch 975/999
----------
train Loss: 0.0085 Acc: 1.0000
val Loss: 1.3549 Acc: 0.8142
Epoch 976/999
----------
train Loss: 0.0332 Acc: 0.9877
val Loss: 1.3727 Acc: 0.8142
Epoch 977/999
----------
train Loss: 0.0201 Acc: 0.9877
val Loss: 1.3794 Acc: 0.8142
Epoch 978/999
----------
train Loss: 0.0159 Acc: 1.0000
val Loss: 1.3654 Acc: 0.8142
Epoch 979/999
----------
train Loss: 0.0023 Acc: 1.0000
val Loss: 1.3228 Acc: 0.8074
Epoch 980/999
----------
train Loss: 0.0004 Acc: 1.0000
val Loss: 1.3208 Acc: 0.8074
Epoch 981/999
----------
train Loss: 0.0007 Acc: 1.0000
val Loss: 1.3214 Acc: 0.8074
Epoch

import matplotlib.pyplot as plt
scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

ohist = []
shist = []

ohist = [h.cpu().numpy() for h in hist]
shist = [h.cpu().numpy() for h in scratch_hist]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.figure(figsize=(10,10))
plt.show()

In [ ]:
#Create plot of Accuarcy train and validation. 
plt.xlabel("Training Epoche")
plt.ylabel("Loss rate")
plt.plot(range(1,num_epochs+1),loss_train,label="Training Pretrained")
plt.plot(range(1,num_epochs+1),loss_val,label="Validation")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.rcParams["figure.figsize"] = (20,3)
plt.show()